<a href="https://colab.research.google.com/github/royorel/Lifespan_Age_Transformation_Synthesis/blob/master/LATS_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lifespan Age Transformation Synthesis Demo

This Colab notebook demonstrates the capabilities of the GAN architecture proposed in our paper. 

This colab lets you try our method on your own image!

First, let's download the github repository and install all dependencies.

In [2]:
!git clone https://github.com/royorel/Lifespan_Age_Transformation_Synthesis
%cd Lifespan_Age_Transformation_Synthesis/
!pip3 install -r requirements.txt

Cloning into 'Lifespan_Age_Transformation_Synthesis'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 520 (delta 1), reused 3 (delta 1), pack-reused 512
Receiving objects: 100% (520/520), 22.73 MiB | 15.15 MiB/s, done.
Resolving deltas: 100% (320/320), done.
/content/Lifespan_Age_Transformation_Synthesis
     |████████████████████████████████| 686kB 6.7MB/s 
     |████████████████████████████████| 245kB 18.9MB/s 
     |████████████████████████████████| 204kB 21.6MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp36-none-any.whl size=655250 sha256=40dd258af15434632d51e5c56ef7cd668e190c6609339cfc9e7c7d22d10eaa8a
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5712 sha256=8ffc5e75c71e1cb3e5f4ddc7d8b711af82b7b1c1c4d5f29e11259ed76186838e
  Stored in d

Now let's download the pretrained models for males and females.

In [3]:
!python download_models.py

Extracting males model zip file
Done!
Extracting females model zip file
Done!


Here, we import libraries and set options.

In [1]:
import os
from collections import OrderedDict
from options.test_options import TestOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer

opt = TestOptions().parse(save=False)
opt.display_id = 0 # do not launch visdom
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip
opt.in_the_wild = True # This triggers preprocessing of in the wild images in the dataloader
opt.traverse = True # This tells the model to traverse the latent space between anchor classes
opt.interp_step = 0.05 # this controls the number of images to interpolate between anchor classes

/content/Lifespan_Age_Transformation_Synthesis
------------ Options -------------
activation: lrelu
batchSize: 1
checkpoints_dir: ./checkpoints
compare_to_trained_class: 1
compare_to_trained_outputs: False
conv_weight_norm: True
dataroot: ./datasets/males/
debug_mode: False
decoder_norm: pixel
deploy: False
display_id: 1
display_port: 8097
display_single_pane_ncols: 6
display_winsize: 256
fineSize: 256
full_progression: False
gen_dim_per_style: 50
gpu_ids: [0]
how_many: 50
id_enc_norm: pixel
image_path_file: None
in_the_wild: False
input_nc: 3
interp_step: 0.5
isTrain: False
loadSize: 256
make_video: False
max_dataset_size: inf
nThreads: 4
n_adaptive_blocks: 4
n_downsample: 2
name: debug
ngf: 64
no_cond_noise: False
no_flip: False
no_moving_avg: False
normalize_mlp: True
ntest: inf
output_nc: 3
phase: test
random_seed: -1
resize_or_crop: resize_and_crop
results_dir: ./results/
serial_batches: False
sort_classes: True
sort_order: ['0-2', '3-6', '7-9', '15-19', '30-39', '50-69']
trained_

usage: ipykernel_launcher.py [-h] [--name NAME] [--gpu_ids GPU_IDS]
                             [--checkpoints_dir CHECKPOINTS_DIR]
                             [--batchSize BATCHSIZE] [--loadSize LOADSIZE]
                             [--fineSize FINESIZE] [--input_nc INPUT_NC]
                             [--output_nc OUTPUT_NC] [--dataroot DATAROOT]
                             [--sort_classes SORT_CLASSES]
                             [--sort_order SORT_ORDER]
                             [--resize_or_crop RESIZE_OR_CROP]
                             [--serial_batches] [--no_flip]
                             [--nThreads NTHREADS]
                             [--max_dataset_size MAX_DATASET_SIZE]
                             [--display_single_pane_ncols DISPLAY_SINGLE_PANE_NCOLS]
                             [--display_winsize DISPLAY_WINSIZE]
                             [--display_port DISPLAY_PORT]
                             [--display_id DISPLAY_ID]
                         

Next on, we call the data loader and the visualizer class that generates the video from the network outputs.

In [17]:
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
visualizer = Visualizer(opt)

AgingDataLoader
dataset [MulticlassUnalignedDataset] was created


Here, we define our model.

NOTE: if you plan to try the method for a female, change opt.name to 'females_model'.

In [3]:
opt.name = 'males_model' # change to 'females model' if you're trying the code on a female image
model = create_model(opt)
model.eval()

Generator(
  (id_encoder): IdentityEncoder(
    (encoder): Sequential(
      (0): ReflectionPad2d((3, 3, 3, 3))
      (1): EqualConv2d(
        (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
      )
      (2): PixelNorm()
      (3): ReLU(inplace=True)
      (4): ReflectionPad2d((1, 1, 1, 1))
      (5): EqualConv2d(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
      )
      (6): PixelNorm()
      (7): ReLU(inplace=True)
      (8): ReflectionPad2d((1, 1, 1, 1))
      (9): EqualConv2d(
        (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2))
      )
      (10): PixelNorm()
      (11): ReLU(inplace=True)
      (12): ResnetBlock(
        (conv_block): Sequential(
          (0): ReflectionPad2d((1, 1, 1, 1))
          (1): EqualConv2d(
            (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
          )
          (2): PixelNorm()
          (3): ReLU(inplace=True)
          (4): ReflectionPad2d((1, 1, 1, 1))
          (5): EqualConv2d(

InferenceModel(
  (netG): Generator(
    (id_encoder): IdentityEncoder(
      (encoder): Sequential(
        (0): ReflectionPad2d((3, 3, 3, 3))
        (1): EqualConv2d(
          (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
        )
        (2): PixelNorm()
        (3): ReLU(inplace=True)
        (4): ReflectionPad2d((1, 1, 1, 1))
        (5): EqualConv2d(
          (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
        )
        (6): PixelNorm()
        (7): ReLU(inplace=True)
        (8): ReflectionPad2d((1, 1, 1, 1))
        (9): EqualConv2d(
          (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2))
        )
        (10): PixelNorm()
        (11): ReLU(inplace=True)
        (12): ResnetBlock(
          (conv_block): Sequential(
            (0): ReflectionPad2d((1, 1, 1, 1))
            (1): EqualConv2d(
              (conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
            )
            (2): PixelNorm()
            (3): ReLU(inpl

OK, it's time to upload your image.

For best results, use images according to the following guidelines:

1. The image should contain a single face.
2. Image was taken from a digital camera (phone cameras are fine). Old images from film cameras would produce low quality results.
3. Pure RGB images only. No black & white, grayscale, sepia, or filtered images (e.g. Instagram filters).
4. Person's head should directly face the camera. Looking sideways/downwards/upwards degrades the results.
5. The person's face should not be occluded (or partially occluded) by any item.
6. Both eyes should be open and visible. (eyeglasses are ok, no sunglasses)

In [4]:
# upload your image (the code supports only a signle image at a time)
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
  img_path = filename
  print('User uploaded file "{name}"'.format(name=filename))

Saving lucas.jpg to lucas (3).jpg
User uploaded file "lucas.jpg"


Finally, we preprocess the image, run the network, and save the result.

In [21]:
data = dataset.dataset.get_item_from_path(img_path)
visuals = model.inference(data)

os.makedirs('results', exist_ok=True)
out_path = os.path.join('results', img_path[:-4] + '.mp4')
visualizer.make_video(visuals, out_path)

Let's display at the results.

NOTE: if you're using chrome, uncomment the lines below. For some reason, mp4 files won't display on chrome browser, so we need to convert to webm.

In [53]:
use_webm = False
# For some unknown reason the mp4 video is not displayed on chrome
# If you have chrome, uncomment the following lines to convert the 
# result to webm for display purposes

# !pip3 install webm
# webm_out_path = os.path.join('results', img_path[:-4] + '.webm')
# !webm -i $out_path $webm_outpath
# use_webm = True

from IPython.display import HTML
from base64 import b64encode
video_path = webm_out_path if use_webm else out_path
video_type = "video/webm" if use_webm else "video/mp4"
mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=256 controls>
      <source src="{0}" type="{1}">
</video>
""".format(data_url, video_type))

You can download the result if you want

In [24]:
files.download(out_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>